In [1]:
!uv pip install navec pymorphy2 pandas

Audited 3 packages in 6ms


In [2]:
from navec import Navec
import pandas as pd
import pymorphy2

# существительные, нам понадобиться только колонка с начальной формой – bare
nouns = pd.read_csv('nouns.csv', sep='\t')[['bare']]
# переименуем колонку под наше обозначение
nouns = nouns.rename(columns={"bare": "word"})

# эмбеддинги, файл доступен в репозитории библиотеки
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

# морфологический анализ слов
morph = pymorphy2.MorphAnalyzer()
nouns

/Users/a.veshutov/.cache/uv/archive-v0/-57Le-KDc7jKwFVN2e-bb/lib/python3.10/site-packages/pymorphy2/analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


,word
0,человек
1,год
2,время
3,рука
4,стать
...,...
26977,ссыкуха
26978,туроператор
26979,валенок
26980,бабло


In [3]:
def is_proper_noun(word):
    parsed = morph.parse(word)[0]
    # есть в датасете эмбеддингов
    if (word not in navec):
        return False
    # фильтруем географические названия
    if ('Geox' in parsed.tag):
        return False
    # фильтруем имена
    if ('Name' in parsed.tag):
        return False
    # фильтруем фамилии
    if ('Surn' in parsed.tag):
        return False
    # убеждаемся, что точно существительное
    return 'NOUN' == parsed.tag.POS

filtered_nouns = nouns[nouns['word'].apply(is_proper_noun)]
# убираем дубликаты
filtered_nouns = filtered_nouns.sort_values('word').drop_duplicates('word', keep='last')
filtered_nouns = filtered_nouns.reset_index(drop=True)

In [4]:
def to_vec(word):
    return navec[word]

filtered_nouns['embeddings'] = filtered_nouns['word'].apply(to_vec)

In [5]:
filtered_nouns

,word,embeddings
0,абажур,"[0.49163, 0.003914773, -0.034092356, -0.338077..."
1,аббат,"[-0.4445479, 0.24872686, -0.16423002, -0.00452..."
2,аббатиса,"[-0.17017908, 0.67665476, 0.21191145, -0.25886..."
3,аббатство,"[-0.089431904, 0.39096224, -0.19686256, 0.3650..."
4,аббревиатура,"[0.036720417, 0.92295927, 0.37911993, -0.89247..."
...,...,...
19869,ёжик,"[0.5135839, 0.48293516, 0.51696765, 0.1938662,..."
19870,ёлка,"[0.57082283, -0.02571172, 0.20820206, -0.17096..."
19871,ёлочка,"[0.7598189, 0.5453649, -0.112766445, -0.277389..."
19872,ёмкость,"[0.22543699, -0.39721358, 0.6805563, 0.3375504..."


In [6]:
# если хочется выбрать конкретное
secret_word = filtered_nouns[filtered_nouns['word'] == 'программист']
# secret_word = filtered_nouns.sample(n=1)
secret_word

,word,embeddings
13684,программист,"[-0.62495345, -0.37000862, 0.29700223, -0.0212..."


In [7]:
import math

def cosine_similarity_manual(vec1, vec2):
    dot_product = sum(v1 * v2 for v1, v2 in zip(vec1, vec2))
    magnitude_vec1 = math.sqrt(sum(v1**2 for v1 in vec1))
    magnitude_vec2 = math.sqrt(sum(v2**2 for v2 in vec2))

    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0

    return dot_product / (magnitude_vec1 * magnitude_vec2)
  
# берем вектор секретного слова
secret_word_vec = secret_word['embeddings'].values[0]
# считаем косинусное сходство с ним для всех остальных слов
filtered_nouns['cosine_similarity'] = filtered_nouns['embeddings'].apply(lambda emb: cosine_similarity_manual(secret_word_vec, emb))
# сортируем по полученному значению
filtered_nouns = filtered_nouns.sort_values(by='cosine_similarity', ascending=False)
filtered_nouns = filtered_nouns.reset_index(drop=True)

In [8]:
filtered_nouns

,word,embeddings,cosine_similarity
0,программист,"[-0.62495345, -0.37000862, 0.29700223, -0.0212...",1.000000
1,компьютерщик,"[-0.44697946, 0.33564645, 0.11986674, -0.13594...",0.687300
2,хакер,"[-0.5767624, 0.4216987, 0.3996202, -0.14531006...",0.685922
3,электронщик,"[-0.44697946, 0.33564645, 0.11986674, 0.072180...",0.568976
4,менеджер,"[-0.3132882, -0.15921138, 0.13000736, 0.259968...",0.561805
...,...,...,...
19869,острота,"[0.5788273, -0.27504855, -0.5113364, 0.3327470...",-0.206003
19870,процессия,"[0.68426675, -0.2751317, -0.14503816, -0.28942...",-0.212166
19871,конфискация,"[0.3030738, 0.8606349, -0.017849343, -0.559853...",-0.212496
19872,подкладка,"[0.17746952, 0.32052407, 0.4761879, -0.1956500...",-0.215849


In [9]:
from datetime import datetime

filtered_nouns['rank'] = filtered_nouns.index + 1
filtered_nouns['date'] = datetime.today().strftime('%Y-%m-%d')

filtered_nouns[['word', 'rank', 'date']].to_csv('words.csv', index=False)

In [10]:
!uv pip install numpy scikit-learn

Audited 2 packages in 3ms


In [11]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# количество введенных слов
N_SAMPLES = 10
# размер батча для поиска решений
BATCH_SIZE = 5000

solution_nouns = filtered_nouns.copy()
embeddings = np.vstack(solution_nouns['embeddings'].to_list())

# узнаем ранг случайных слов в количестве = n_samples
samples = solution_nouns.sample(n=N_SAMPLES)
sample_indices = samples.index.to_numpy()

potential_first = set()

for start in range(0, len(solution_nouns), BATCH_SIZE):
    end = min(start + BATCH_SIZE, len(solution_nouns))
    similarities = cosine_similarity(embeddings[start:end], embeddings)
    print(f"Обработка батча {start}-{end}")
    
    for i, idx in enumerate(range(start, end)):
        sorted_indices = np.argsort(similarities[i])[::-1]
        sample_positions = [np.where(sorted_indices == si)[0][0] for si in sample_indices]
        
        if np.array_equal(np.argsort(sample_indices), np.argsort(sample_positions)):
            potential_first.add(solution_nouns.iloc[idx]['word'])

print("Введенные слова:")
print(samples[['rank', 'word']].values)
print(f"Потенциальные решения за {N_SAMPLES} попыток: {potential_first}")

Обработка батча 0-5000
Обработка батча 5000-10000
Обработка батча 10000-15000
Обработка батча 15000-19874
Введенные слова:
[[17703 'влияние']
 [5244 'кока']
 [18653 'сношение']
 [19161 'негодование']
 [13778 'ленца']
 [15310 'свойство']
 [2740 'стишок']
 [2897 'аврал']
 [2471 'милиционер']
 [18657 'любопытство']]
Потенциальные решения за 10 попыток: {'программист'}


In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_solution(original_nouns, solution_nouns, n_samples, batch_size, random_state):
    """
    Аргументы:
        original_nouns: датасет из которого было загадано слово
        solution_nouns: датасет слов поиска решений
        n_samples: начальное количество вводимых слов
        batch_size: размер батча для поиска решения
        random_state: настройка случайного выбора слов
    """
    embeddings = np.vstack(solution_nouns['embeddings'].to_list())
    
    while True:
        print(f"Пробуем за {n_samples} попыток...")
        
        # узнаем ранг случайных слов в количестве = n_samples
        # random_state позволяет не терять предыдущие слова при инкременте n_samples
        samples = original_nouns.sample(n=n_samples, random_state=random_state)
        sample_indices = samples.index.to_numpy()
        
        potential_first = set()
        
        # находим все потенциальные решения, для которых относительный порядок сохраняется
        for start in range(0, len(solution_nouns), batch_size):
            end = min(start + batch_size, len(solution_nouns))
            print(f"Обабатываем батч {start}-{end}")
            
            # разом считаем косинусное сходство для всего батча потенциальных решений
            similarities = cosine_similarity(embeddings[start:end], embeddings)
            
            for i, idx in enumerate(range(start, end)):
                sorted_indices = np.argsort(similarities[i])[::-1]
                sample_positions = [np.where(sorted_indices == si)[0][0] for si in sample_indices]

                # если относительный порядок сохранился, то добавляем в список потенциальных решений
                if np.array_equal(np.argsort(sample_indices), np.argsort(sample_positions)):
                    potential_first.add(solution_nouns.iloc[idx]['word'])
        
        print(f"Нашли {len(potential_first)} потенциальных решений за {n_samples} попыток")
        
        # выходим, если осталось только 1 решение или исчерпали весь датасет
        if len(potential_first) <= 1 or n_samples >= len(solution_nouns):
            print(samples[['word', 'rank']])
            return n_samples, potential_first
        
        # потенциальных решений > 1, продолжаем
        n_samples += 1

# начальное количество введенных слов
N_SAMPLES = 5
# размер батча для поиска решений
BATCH_SIZE = 5000
# чтобы введенные слова не обновлялись
RANDOM_STATE = 42

solution_nouns = filtered_nouns.copy()
used_samples, potential_first = find_solution(filtered_nouns, solution_nouns, N_SAMPLES, BATCH_SIZE, RANDOM_STATE)

print(f"Решение {potential_first} найдено за {used_samples} попыток")

Пробуем за 5 попыток...
Обабатываем батч 0-5000
Обабатываем батч 5000-10000
Обабатываем батч 10000-15000
Обабатываем батч 15000-19874
Нашли 107 потенциальных решений за 5 попыток
Пробуем за 6 попыток...
Обабатываем батч 0-5000
Обабатываем батч 5000-10000
Обабатываем батч 10000-15000
Обабатываем батч 15000-19874
Нашли 8 потенциальных решений за 6 попыток
Пробуем за 7 попыток...
Обабатываем батч 0-5000
Обабатываем батч 5000-10000
Обабатываем батч 10000-15000
Обабатываем батч 15000-19874
Нашли 2 потенциальных решений за 7 попыток
Пробуем за 8 попыток...
Обабатываем батч 0-5000
Обабатываем батч 5000-10000
Обабатываем батч 10000-15000
Обабатываем батч 15000-19874
Нашли 1 потенциальных решений за 8 попыток
            word   rank
3950         фсб   3951
1272   страдалец   1273
14384    дыхание  14385
18502      ягода  18503
3165        сука   3166
15793  кочерыжка  15794
9796        галс   9797
19589     вражда  19590
Решение {'программист'} найдено за 8 попыток
